In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd

import gc
import timeit
import sys
import os

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
from scipy.optimize import minimize

# Residual Sum of Squares calculations: synthetic data against data for seen deer per hunting hour

In [4]:
# Revoking stored reported data
# https://ipython.org/ipython-doc/rel-0.12/config/extensions/storemagic.html

%store -r data_Averoy
%store -r data_Tingvoll
%store -r data_Surnadal
%store -r data_Sunndal
%store -r data_Vestnes
%store -r data_Laerdal

In [5]:
def sum_squares_seen_deer(
    number_of_years,
    municipality_file,
    empirical_observations,
    compare,
):
    """
    Calculating the minimum residual_sum_of_squares between synthetic data and
    reported data
    """

    # Revoking the targeted municipality data frame created by the synthetic population generator
    save_dir = "synthetic_data/"
    data_frame = pd.read_pickle(save_dir + municipality_file)

    # Defining column to extract values from
    if compare == "before_hunting":
        comp_choice = "tot_pop_bh"
    if compare == "after_hunting":
        comp_choice = "tot_pop_ah"

    RSS_list = []
    for i in range(0, int(len(data_frame) / number_of_years)):
        # Catch the predicted total population sizes for all observation years
        seen_deer_p = data_frame.iloc[i * number_of_years : (i + 1) * number_of_years][
            comp_choice
        ].values

        seen_deer_e = np.array(empirical_observations)

        # Find the scaling factor that minimises RSS and do the scaling
        scaling_factor = np.sum(np.multiply(seen_deer_p, seen_deer_e)) / np.sum(
            np.multiply(seen_deer_p, seen_deer_p)
        )
        seen_deer_p_scaled = scaling_factor * seen_deer_p

        # Find minimum Residual Sum Square value
        RSS_min = np.sum((seen_deer_p_scaled - seen_deer_e) ** 2)

        RSS_list.append([scaling_factor, RSS_min])

    return RSS_list

In [6]:
def make_RSS_frame(municipality_file, RSS_list):

    # Revoking the targeted municipality data frame created by the synthetic population generator
    save_dir = "synthetic_data/"
    data_frame = pd.read_pickle(save_dir + municipality_file)

    # Remove all rows except those where obs_year == 2021
    df_filtered = data_frame[data_frame["obs_year"] == 2021]

    # Remove columns not used
    cols = [
        "f_calves",
        "y_hinds",
        "a_hinds",
        "m_calves",
        "y_stags",
        "a_stags",
        "ws_fc",
        "ws_yh",
        "ws_ah",
        "ws_mc",
        "ws_ys",
        "ws_as",
        "c_yh",
        "c_ah",
    ]
    df_filtered2 = df_filtered.drop(cols, axis=1)

    # Resert index, otherwise pd.concat does not work
    df_filtered2 = df_filtered2.reset_index(drop=True)

    # Add two columns from RSS_list
    df_RSS = pd.concat(
        [df_filtered2, pd.DataFrame(RSS_list, columns=["scaling", "RSS"])], axis=1
    )

    # Sort the frame on the RSS value
    sorted_sum_squares_frame = df_RSS.sort_values(by=["RSS"]).reset_index(drop=True)

    return sorted_sum_squares_frame

In [7]:
def extract_top_hits(
    i,
    sorted_sum_squares_frame,
    filtering_strategy,
    frac_init_pop,
    number_of_top_hits,
):
    """
    Filtering the sum_squares frame based on an assumption about the
    size of the Dec 31 2020 population vs the Dec 31 2006 population,
    and delivering only the number_of_top_hits best fits.
    """

    if filtering_strategy == "uninformed":
        sorted_sum_squares_frame_filtered = sorted_sum_squares_frame

    if filtering_strategy == "informed":
        # Educated guess filtering - can play with these criteria
        sorted_sum_squares_frame_filtered = sorted_sum_squares_frame[
            (
                sorted_sum_squares_frame.tot_pop_ah
                > frac_init_pop[i] * sorted_sum_squares_frame.init_pop
            )
        ].reset_index(drop=True)

    return sorted_sum_squares_frame_filtered[0:number_of_top_hits]

In [8]:
def run_the_show():
    # Using all 6 municipalities
    municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
    data_municipality = [
        data_Averoy,
        data_Tingvoll,
        data_Surnadal,
        data_Sunndal,
        data_Vestnes,
        data_Laerdal,
    ]

    municipality_frame = [
        "df_original_sorted_Averoy.pkl",
        "df_original_sorted_Tingvoll.pkl",
        "df_original_sorted_Surnadal.pkl",
        "df_original_sorted_Sunndal.pkl",
        "df_original_sorted_Vestnes.pkl",
        "df_original_sorted_Laerdal.pkl",
    ]

    compare = "before_hunting"
    filtering_strategy = "informed"
    frac_init_pop = [0.5] * len(municipalities)  # minmum tot_pop 2020/2006 ratio
    number_of_top_hits = 20

    for q in range(len(municipalities)):
        [
            municipality,
            first_year,
            last_year,
            number_of_years,
            years,
            seen_deer_obs,
            seen_deer_obs_outfield,
            seen_deer_obs_infield,
            hinds_per_stag_obs,
            total_harvest,
            fraction_female_calves_harvested,
            fraction_young_hinds_harvested,
            fraction_adult_hinds_harvested,
            fraction_male_calves_harvested,
            fraction_young_stags_harvested,
            fraction_adult_stags_harvested,
            spring_counts,
        ] = data_municipality[q]

        if municipality == "Surnadal":
            RSS_list = sum_squares_seen_deer(
                number_of_years,
                municipality_frame[q],
                seen_deer_obs_infield,
                compare,
            )
        else:
            RSS_list = sum_squares_seen_deer(
                number_of_years,
                municipality_frame[q],
                seen_deer_obs,
                compare,
            )

        sorted_sum_squares_frame = make_RSS_frame(municipality_frame[q], RSS_list)

        top_hits_frame_filtered = extract_top_hits(
            q,
            sorted_sum_squares_frame,
            filtering_strategy,
            frac_init_pop,
            number_of_top_hits,
        )

        # Storing top hits frames
        save_dir = "synthetic_data/"
        top_hits_frame_filtered.to_pickle(
            save_dir + "top_hits_" + municipality + "_seen_deer" + ".pkl"
        )

        print("Municipality = ", municipality)
        display(top_hits_frame_filtered)
        print()
        print(
            "Percentage difference between min and max RSS value: ",
            round(
                (
                    (
                        top_hits_frame_filtered["RSS"].max()
                        - top_hits_frame_filtered["RSS"].min()
                    )
                    / top_hits_frame_filtered["RSS"].min()
                )
                * 100,
                2,
            ),
        )
        print()
        print("Mean column values of top_hits_frame:")
        display(top_hits_frame_filtered.mean())
        print()
        print()

In [9]:
%%time
# Running all scripts by calling up run_the_show
# This is done to get rid of memory leaks
run_the_show()

Municipality =  Averoy


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,scaling,RSS
0,1400,1.4,0.6,147,161,483,147,221,239,2021,1186,1597,1164,1.4739,1.1488,15.1156,54,0.000740,0.632886
1,1450,1.3,0.7,166,138,492,166,145,339,2021,1217,1563,1130,1.2831,0.9564,20.0556,84,0.000738,0.658284
2,1300,1.8,0.6,130,120,548,130,66,304,2021,1084,1441,1008,1.3521,0.9869,16.3239,54,0.000758,0.661097
3,1400,1.3,0.8,154,129,487,154,123,351,2021,1309,1662,1229,1.1868,0.8837,20.9070,104,0.000710,0.671749
4,1150,2.0,0.5,120,96,508,120,127,175,2021,1108,1528,1095,1.6428,1.3009,13.2823,39,0.000774,0.673681
5,1600,1.4,0.5,264,143,481,264,89,357,2021,1259,1704,1271,1.6231,1.3226,18.5120,65,0.000738,0.685546
6,1200,2.0,0.5,198,91,444,198,53,214,2021,980,1361,928,1.5938,1.1992,10.2493,25,0.000831,0.689359
7,1350,1.4,0.6,168,76,513,168,109,312,2021,950,1252,819,1.4294,0.9895,21.8832,68,0.000813,0.690976
8,1350,1.5,0.7,202,56,510,202,136,241,2021,1126,1450,1017,1.2601,0.9013,20.5847,81,0.000748,0.695316
9,1300,1.3,0.7,175,80,455,175,82,330,2021,1007,1317,884,1.2611,0.8551,18.8661,66,0.000787,0.697476



Percentage difference between min and max RSS value:  12.99

Mean column values of top_hits_frame:


init_pop        1332.500000
init_hps           1.595000
sh_threshold       0.585000
i_fcalves        180.550000
i_yhinds          98.950000
i_ahinds         490.750000
i_mcalves        180.550000
i_ystags         119.800000
i_astags         259.100000
obs_year        2021.000000
spring_pop      1106.950000
tot_pop_bh      1462.350000
tot_pop_ah      1029.350000
hps_bh             1.457130
hps_ah             1.091855
mig%              19.725580
num_migs          68.150000
scaling            0.000770
RSS                0.692259
dtype: float64



Municipality =  Tingvoll


...



Municipality =  Laerdal


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,scaling,RSS
0,1500,1.7,0.5,165,103,633,165,225,208,2021,1236,1697,1165,1.6987,1.8299,15.6815,53,0.000123,0.018216
1,1750,1.3,0.6,201,99,662,201,105,480,2021,1371,1891,1359,1.5750,1.6332,6.4699,26,0.000116,0.018371
2,1650,1.4,0.6,181,97,653,181,246,289,2021,1290,1728,1196,1.4723,1.4946,11.8196,45,0.000115,0.018376
3,1850,1.4,0.7,249,189,598,249,225,337,2021,1510,2023,1491,1.3350,1.3128,10.0570,50,0.000108,0.018378
4,1650,1.3,0.5,165,74,671,165,309,264,2021,1475,2066,1534,1.6637,1.7496,9.0927,35,0.000117,0.018435
5,1650,1.7,0.5,198,86,702,198,213,250,2021,1380,1882,1350,1.6595,1.7535,13.4585,50,0.000116,0.018514
6,1550,1.7,0.5,170,114,647,170,205,241,2021,1206,1678,1146,1.6753,1.7959,10.5254,33,0.000124,0.018524
7,1700,1.6,0.5,170,100,736,170,251,272,2021,1273,1772,1240,1.6683,1.7773,12.2701,43,0.000119,0.018552
8,1450,2.0,0.5,188,164,550,188,157,200,2021,1471,2064,1532,1.6962,1.7958,10.7679,42,0.000120,0.018586
9,1500,2.0,0.5,202,146,584,202,160,204,2021,1278,1794,1262,1.6733,1.7818,7.3419,24,0.000118,0.018598



Percentage difference between min and max RSS value:  3.39

Mean column values of top_hits_frame:


init_pop        1597.500000
init_hps           1.625000
sh_threshold       0.530000
i_fcalves        184.250000
i_yhinds         125.300000
i_ahinds         629.050000
i_mcalves        184.250000
i_ystags         192.500000
i_astags         279.650000
obs_year        2021.000000
spring_pop      1324.400000
tot_pop_bh      1832.100000
tot_pop_ah      1300.100000
hps_bh             1.623885
hps_ah             1.710445
mig%              10.433030
num_migs          38.150000
scaling            0.000119
RSS                0.018604
dtype: float64



CPU times: user 3h 49min 9s, sys: 13min 42s, total: 4h 2min 51s
Wall time: 4h 30s
